# Question word prediction

> Group 12: Tristan Perrot & Romain Darous

Task is to train and evaluate a **char per char Transformer model** model using any available QA-corpus, for instance, the [SQuAD corpus](https://rajpurkar.github.io/SQuAD-explorer/).


METTRE EN CONTEXTE LE DEBUT DE LA QUESTION EN CORRIGEANT LE CODE DEJA FAIT
ADAPTER POUR AUGMENTER LE NOMBRE DE TRANSFORMERS EVENTUELLEMENT ET FAIRE PLUSIEURS COUCHES

# 0. Importing modules

In [1]:
import json
import math
import os

# Importing
import matplotlib.pyplot as plt
import requests
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm, trange
import random
from datetime import datetime
import numpy as np

# Importing models
import char_dataset
import cpc_model


/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


# 1. Data pre-processing

## 1.1. Loading the dataset
**Note :** we only want to be able te recover the beginning of a question. For that, it doesn"t matter whether the question is impossible to answer or not.

In [2]:
data_dir = 'data'
if data_dir not in os.listdir():
    os.mkdir(data_dir)

if "squad_train.json" not in os.listdir(data_dir):
    # Download data at https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
    res = requests.get(
        "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json")
    data = json.loads(res.text)

    # Save data to file
    with open(data_dir + "/squad_train.json", "w") as f:
        json.dump(data, f)

with open(data_dir + "/squad_train.json", "r") as f:
    data = json.load(f)

# Extract answer text and question text
answers = []
questions = []
for article in data["data"]:
    for paragraph in article["paragraphs"]:
        for qa in paragraph["qas"]:             
            if qa["is_impossible"] and len(qa["question"]) > 4:
                answers.append("")
            else :
                answers.append(qa["answers"][0]['text'])
            questions.append(qa["question"])
            

print("Number of questions:", len(questions))

# Print some examples
for i in range(5):
    print()
    print("Question:", questions[i])
    print("Answer:", answers[i])

Number of questions: 130319

Question: When did Beyonce start becoming popular?
Answer: in the late 1990s

Question: What areas did Beyonce compete in when she was growing up?
Answer: singing and dancing

Question: When did Beyonce leave Destiny's Child and become a solo singer?
Answer: 2003

Question: In what city and state did Beyonce  grow up? 
Answer: Houston, Texas

Question: In which decade did Beyonce become famous?
Answer: late 1990s


In [3]:
print(questions[i] + ' ' + answers[i])

In which decade did Beyonce become famous? late 1990s


## 1.2. Making a suitable dataset
``<BOS>`` token. Indicates that the sentence is starting.

We will make the prediction of the sentence in reverse mode, as we want to predict the beginning of a question. We will use unidirectionnal attention as well.

In [4]:
# Concatenating questions and answers
dataset = [(questions[i].lower() + ' ' + answers[i].lower())[::-1] for i in range(len(questions))]

# Shuffle dataset
random.shuffle(dataset)

# Splitting into train, validation, and test sets
train_dataset, test_dataset = train_test_split(dataset, test_size = int(0.1*len(dataset)), train_size=int(0.9*len(dataset)))
train_dataset, val_dataset = train_test_split(train_dataset, train_size=int(0.85*len(train_dataset)), test_size = int(0.15*len(train_dataset)))

In [5]:
print(f"Size of the dataset : {len(dataset)}")
print(f"Size of the train, val and test sets : {len(train_dataset), len(val_dataset), len(test_dataset)}")
print(f"Example of original datapoint : {questions[0] + ' ' + answers[0]}")
print(f"Example of formatted datapoint : {dataset[0]}")

Size of the dataset : 130319
Size of the train, val and test sets : (99693, 17593, 13031)
Example of original datapoint : When did Beyonce start becoming popular? in the late 1990s
Example of formatted datapoint :  ?sevitavresnoc eht fo citsiretcarahc gninifed a si tahw


## 1.3. Building a character dataset
The dataset will be built in the ``train_charlm`` function as it depends on the desired configuration. The function that allow to build the dataset and their context are provided in the file ``char_dataset.py``.

# 2. The model

In [6]:
# Updating models# Delete the modules from the namespace
del char_dataset
del cpc_model

# Unload the modules from memory
import sys
del sys.modules['char_dataset']
del sys.modules['cpc_model']

# Importing models
import char_dataset
import cpc_model

# Initalizing the vocabulary
char_dataset.CharDataset("init", 0)
print(char_dataset.CharDataset.char_to_id)
print(char_dataset.CharDataset.char_to_id)


{'<PAD>': 0, '<BOQ>': 1, '<UNK>': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, ' ': 29, ',': 30, '.': 31, '-': 32, ';': 33, ':': 34, '!': 35, '?': 36, '0': 37, '1': 38, '2': 39, '3': 40, '4': 41, '5': 42, '6': 43, '7': 44, '8': 45, '9': 46}
{'<PAD>': 0, '<BOQ>': 1, '<UNK>': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, ' ': 29, ',': 30, '.': 31, '-': 32, ';': 33, ':': 34, '!': 35, '?': 36, '0': 37, '1': 38, '2': 39, '3': 40, '4': 41, '5': 42, '6': 43, '7': 44, '8': 45, '9': 46}


## 2.0. Hyperparameters

In [7]:
# ============= Hyper-parameter class for training ============== #

class Config :
    def __init__(self, seq_ctxt = True, MAXLEN = 32) : 
        self.number_of_transformer_encoders = 2
        self.number_of_attention_heads = 2
        self.hidden_size = 64
        self.dropout_prob = 0.1
        self.batch_size = 64
        self.learning_rate = 0.0003
        self.weight_decay = 0.000001
        self.no_of_epochs = 100
        self.is_causal = True # When True, the attention is causal
        self.seq_ctxt = seq_ctxt # When False, forces the context to take the beginning of the answer into account
        self.MAXLEN = MAXLEN

## 2.1. Training
Defining functions for training and testing models.

In [8]:
def train_charlm(config, device) :
    start_time = datetime.now()

    # ==================== Building datasets ================ #
    train_char_set = char_dataset.CharDataset(train_dataset, config.MAXLEN, 
                                              seq_ctxt=config.seq_ctxt)
    val_char_set = char_dataset.CharDataset(val_dataset, config.MAXLEN, 
                                            seq_ctxt=config.seq_ctxt)

    # ======================= Training ======================= #

    

    training_loader = DataLoader(train_char_set, batch_size=config.batch_size)
    validation_loader = DataLoader(val_char_set, batch_size=config.batch_size)

    charlm = cpc_model.CharLM( config, len(char_dataset.CharDataset.id_to_char), 
                              config.MAXLEN, config.is_causal).to(device)
    criterion = nn.CrossEntropyLoss()
    charlm_optimizer = optim.Adam( charlm.parameters(), lr=config.learning_rate )


    patience = 5
    best_val_loss = float('inf')

    charlm.train()
    print( datetime.now().strftime("%X"), "Training starts" )

    iteration = 0
    for epoch in tqdm(range(config.no_of_epochs)) :
        val_loss = 0.0
        for input_tensor, label in training_loader :
            input_tensor, label = input_tensor.to(device), label.to(device)
            charlm_optimizer.zero_grad()
            logits = charlm(input_tensor).to(device)
            loss = criterion(logits.squeeze(1), label)
            loss.backward()
            charlm_optimizer.step()
        iteration += 1

        print( datetime.now().strftime("%X"), "End of epoch", epoch+1, ", loss=", loss.detach().item())
        
        # Validation phase with Early Stopping
        charlm.eval()
        
        with torch.no_grad():
            for input_tensor, label in validation_loader:
                input_tensor, label = input_tensor.to(device), label.to(device)
                logits = charlm(input_tensor).to(device)
                loss = criterion(logits.squeeze(1), label)
                val_loss += loss.item()

        val_loss /= len(validation_loader)

        # Check early stopping condition
        if val_loss <= best_val_loss:
            best_val_loss = val_loss
            patience = 5
        else:
            patience -= 1
            if patience == 0:
                print("Early stopping at epoch", epoch + 1)
                break
        print(datetime.now().strftime("%X"), "Validation loss=", val_loss, "Patience=", patience)

        charlm.train()
    
    end_time = datetime.now()

    return charlm, iteration + 1, end_time - start_time


In [9]:
def test_charlm(model, config, device) :

    # Computing output on the test set
    test_char_set = char_dataset.CharDataset(test_dataset, config.MAXLEN, seq_ctxt=config.seq_ctxt)
    test_loader = DataLoader(test_char_set, batch_size=config.batch_size)

    accuracies = []
    model.to(device)
    model.eval()    
    with torch.no_grad():
        for input_tensor, label in test_loader:
            input_tensor, label = input_tensor.to(device), label.to(device)
            logits = model(input_tensor).to(device)
            _, pred_label = logits.topk(1, dim=-1)
            pred_label = pred_label.squeeze(-1)  # Squeeze to remove extra dimension
            accuracy = (pred_label == label).sum()/len(test_loader)  # Compute accuracy for this batch
            accuracies.append(accuracy.item())  # Append batch accuracy to list)
    total_acc = np.round(np.mean(np.array(accuracies)*100), 2)
    print(f"Test accuracy : {total_acc} %")
    return total_acc

## 2.2. Grid search on parameters

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print( "Running on", device )

Running on cuda


In [ ]:
output = './output/'
model_names = ['charlm_seq_32_es', 'charlm_seq_64_es', 
               'charlm_nseq_32_es', 'charlm_nseq_64_es']
configs = [Config(seq_ctxt=True, MAXLEN=32), Config(seq_ctxt=True, MAXLEN=64),
           Config(seq_ctxt=False, MAXLEN=32), Config(seq_ctxt=False, MAXLEN=64)]
test_acc = []
models = []
train_time = []
epochs = []

for config, model_name in zip(configs, model_names) :
    # Training the model
    model, epoch, delta = train_charlm(config, device)
    models.append(model)
    train_time.append(delta)
    epochs.append(epoch)


05:45:38 Training starts


  0%|          | 0/100 [00:00<?, ?it/s]

05:46:33 End of epoch 1 , loss= 0.167288139462471
05:46:37 Validation loss= 1.0499531774975808 Patience= 5
05:47:32 End of epoch 2 , loss= 0.15651434659957886
05:47:36 Validation loss= 0.9790533195536044 Patience= 5
05:48:30 End of epoch 3 , loss= 0.19782622158527374
05:48:34 Validation loss= 0.9477244002764301 Patience= 5
05:49:28 End of epoch 4 , loss= 0.13825955986976624
05:49:32 Validation loss= 0.9231590684226882 Patience= 5
05:50:26 End of epoch 5 , loss= 0.16033703088760376
05:50:30 Validation loss= 0.9068655153256002 Patience= 5
05:51:24 End of epoch 6 , loss= 0.14073769748210907
05:51:28 Validation loss= 0.895369708201068 Patience= 5
05:52:22 End of epoch 7 , loss= 0.13588538765907288
05:52:26 Validation loss= 0.8896045930920558 Patience= 5
05:53:20 End of epoch 8 , loss= 0.13501837849617004
05:53:24 Validation loss= 0.8811629133384556 Patience= 5
05:54:19 End of epoch 9 , loss= 0.09646333009004593
05:54:23 Validation loss= 0.8742896638889616 Patience= 5
05:55:17 End of epoch 

In [ ]:
for model, config, model_name in zip(models, configs, model_names) :
    # Test accuracy
    accuracy = test_charlm(model, config, device)
    test_acc.append(accuracy)
    

In [ ]:
for model, model_name, epoch in zip(models, model_names, epochs) :
    # Saving the model
    torch.save(model.state_dict(), f"{output}{model_name}{epoch}.pth")
    print("Model saved successfully")

Building a ``.csv`` file to store the results :

In [ ]:
import pandas as pd

data = {
    'Running time' : train_time,
    'Last epoch' : epochs,
    'Type of context' : ['Sequential' if cfg.seq_ctxt else 'Split' for cfg in configs],
    'Context size' : [cfg.MAXLEN for cfg in configs],
    'Test accuracy (%)' : test_acc
}

# Create the DataFrame with row names
df = pd.DataFrame(data, index=model_names)
df.to_csv(output + 'metrics.csv')

## 2.2. Loading a model
Now that we trained several models, we can load them.

In [ ]:
"""# Load a model
charlm = cpc_model.CharLM( config, len(char_dataset.CharDataset.id_to_char), config.MAXLEN, config.is_causal).to(device)
charlm.load_state_dict(torch.load('./output/charlm_seq_model_early_stopping.pth'))
charlm.eval()"""

# Getting the best model
best_model_idx = np.argmax(test_acc)
charlm = models[best_model_idx]
config = configs[best_model_idx]
print(f"Best model : {model_names[best_model_idx]} : \n\n", df.loc[model_names[best_model_idx]])

## 2.2. Evaluation on the best model

## 2.3. User interaction

In [ ]:
# ==================== User interaction ==================== #
while True:
    text = input("> ").strip()
    if text == "" :
        continue
    elif text == "QUIT" :
        break

    # Will be used to output question
    full_question = list(text.lower())[::-1]

    if full_question[-1] != ' ' : 
        full_question.append(' ')
    if '?' not in full_question :
        full_question = ['?'] + full_question
        
    new_character = full_question[-1]

    # Recovering the beginning of the question
    try :
        count = 0
        MAX_COUNT = 50
        while new_character != char_dataset.CharDataset.BOQ and count < MAX_COUNT :
            # Building context
            char_list = []
            if config.seq_ctxt :
                char_list = full_question[-config.MAXLEN:]
            else :
                tmp = "".join(full_question)
                words_a, words_q = tmp.split("?")[0], tmp.split("?")[1]
                words_a = list(words_a)
                len_q = len(words_q) # counting the <BOS> token
                if len_q <= config.MAXLEN//2 or len(words_a) < config.MAXLEN//8 :
                    char_list = full_question[-config.MAXLEN:]
                else :
                    char_list = words_a[-config.MAXLEN//2:] + full_question[-config.MAXLEN//2:]

            ctxt = [char_dataset.CharDataset.char_to_id[c] if c in char_dataset.CharDataset.char_to_id 
                    else char_dataset.CharDataset.char_to_id[char_dataset.CharDataset.char_to_id[char_dataset.CharDataset.UNK]] 
                    for c in char_list]            
            
            # Computing the next character
            input_tensor = torch.tensor( [0]*(config.MAXLEN-len(ctxt)) + ctxt).unsqueeze(0).to(device)
            logits = charlm(input_tensor).squeeze().to(device)
            _, new_character_tensor = logits.topk(1)
            id = new_character_tensor.detach().item()
            new_character = char_dataset.CharDataset.id_to_char[id]
            
            # Uploading the final output
            full_question.append(new_character)
            count += 1

        full_question = "".join(full_question[::-1])
        print(f"Recovered question : {full_question}")
    except KeyError :
        print("ERROR")
        raise KeyError
        continue